In [70]:
import cv2

In [71]:
import face_recognition

In [72]:
video_capture = cv2.VideoCapture(0)

In [73]:
# Load a sample picture and learn how to recognize it.
dheeraj_image = face_recognition.load_image_file("dheeraj.jpg")
dheeraj_encoding = face_recognition.face_encodings(dheeraj_image)[0]

In [74]:
# Create arrays of known face encodings and their names
known_face_encodings = [
    dheeraj_encoding
]
known_face_names = [
    "Dheeraj"    
]


In [78]:
video_capture = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [43]:
from imutils import contours

In [44]:
import numpy as np

In [45]:
import imutils

In [46]:
image = cv2.imread("paper1.JPG")
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
 

In [47]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 1: Edge Detection


In [48]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]


In [49]:
from transform import four_point_transform

In [50]:
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
      
# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


STEP 2: Find contours of paper


In [16]:
from skimage.filters import threshold_adaptive

In [51]:
import util

In [52]:
show=True
if screenCnt.__len__() != 0:
    if show:
        cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
        cv2.imwrite('outlined.jpg', image)
        cv2.imshow("Outline", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
else:
    warped = orig

warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
warped = threshold_adaptive(warped, 251, offset=10)
warped = warped.astype("uint8") * 255

if show:
    cv2.imshow("Original", util.resize(orig, height=650))
    cv2.imshow("Scanned", util.resize(warped, height=650))
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

/home/dheeraj/work/env1/lib/python3.6/site-packages/skimage/filters/thresholding.py:221: skimage_deprecation: Function ``threshold_adaptive`` is deprecated and will be removed in version 0.15. Use ``threshold_local`` instead.
  def threshold_adaptive(image, block_size, method='gaussian', offset=0,
/home/dheeraj/work/env1/lib/python3.6/site-packages/skimage/filters/thresholding.py:223: UserWarning: The return value of `threshold_local` is a threshold image, while `threshold_adaptive` returned the *thresholded* image.
  warn('The return value of `threshold_local` is a threshold image, while '


In [53]:
import pytesseract
from PIL import Image
from pytesseract import image_to_string


In [54]:
try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract

In [55]:
cv2.imwrite('scan_res.jpg', warped)
text = pytesseract.image_to_string(Image.open('scan_res.jpg'), config="config")

print(text)

ae eee

eet Restaurant
Main, > jusiness Drive
Suite 528
paso Alto California 9430
575-1628095

Fri 04/07/2017 3:57 PM

Order ID: #4a59c18f
Order Number: 1

ements swede em ke hee a

Chocolate Chip Cooki 5.00

Apple Pie 3.00
Lava Cake 4.00
Sub Total «USD$ 12.00
Grand Total: oo $12.00

Card; RAKKXXXXXXXO 14.16
Tip — | | 2.16

em mek em hie tk fb DS D6 ft hho BD 2D Oo b® 6.S Sb ABO Ah


In [123]:
video_capture = cv2.VideoCapture(0)
padding = 100
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    height, width, channels = rgb_frame.shape

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        
        
        h=top-bottom
        
        if h>height:
            h=width-bottom
        
#         roi = im[y1:y2, x1:x2]
        crop_img = frame[bottom:, left:right-left]
#         cv2.rectangle(frame, (left, bottom), (right, bottom+height), (0, 255, 0), cv2.FILLED)

        text = pytesseract.image_to_string(crop_img, config="config")
        if text !="":
            print(text)
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

#         # Draw a label with a name below the face
#         cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

»
Ne
a
eet\\
“Sa
dine
a
sy
Say

Hist
Yuval Noah

a!

*
mt

me A Brie!
Yuval N

Saf
A!

antes
Yuval!
m6 (6
ap

; ABr
Histol
of

Sap!
ABi

nea
.
~
Sat
Sal
Sa
4
54
Say
Sa
24
Sa
Yuval
BS yova
-)

B yoval
\


In [121]:
video_capture.release()
cv2.destroyAllWindows()

In [94]:
video_capture = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

#     # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
#     rgb_frame = frame[:, :, ::-1]
    
    ratio = frame.shape[0] / 500.0
    orig = frame.copy()
    frame = imutils.resize(frame, height = 500)
     
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            break

    cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
    
    text = pytesseract.image_to_string(frame, config="config")
    if text !="":
        print(text)


    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

History of
Auman}

Ind
Sapiens
A Brief
History of
Humankind
Sapiens
A Brief
History of
Humankind
*
Sapiens

A Brief
History of
Humankind
*
Sapiens

A Brief

History of
Humankind

 

hy

Bh oi
*
Sapiens

A Briet

History of
Humankind

 

LF

Pr |
a
Sapiens

A Briel
History of

Humankind
Yuval Noah Harart

.
Sapiens
A Brief
History of

Humankind
Yuval Noah Harart

a
Sapiens

A Brief

History of

Humankind
Yuval Noah Harari

.
Sapiens

A Brief

History of
Humankind
Yuval Noah Harari

*
Sapiens

A Brief

History of
Humankind
Yuval Noah Harari

®
Sapiens

\ Brief

History of

Humankind
Yuval Noah Harari
A Brief

History of
I tUmank yg
Yuval Noah Harari

*
Sapiens
A Brief
History of

Ind

   

Auman,
Nivel Noah Harari

+
Sapiens

A Brief
History of

   

Humank ed
Yuval Noah Harari

+
Sapiens

A Brief
History of

Ikd

Tuman
